In [4]:
import pandas as pd
import numpy as np

# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Intro to SQL
Week 5 | Day 2

### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Connect to a local or remote database using Python or Pandas
- Connect to a local or remote database using SQLite Manager (for SQLite) or Postico(for POSTGRES)
- Perform queries using SELECT
- Perform simple aggregations COUNT, MAX/MIN/SUM

## SQLite

SQLite is an embedded SQL database engine. Unlike most other SQL databases, SQLite does not have a separate server process. **SQLite reads and writes directly to ordinary disk files.** A complete SQL database with multiple tables, indices, triggers, and views, is contained in a single disk file.

SQLite is not directly comparable to client/server SQL database engines such as MySQL, Oracle, PostgreSQL, or SQL Server since SQLite is trying to solve a different problem.

SQLite emphasizes economy, efficiency, reliability, independence, and simplicity.

**SQLite does not compete with client/server databases. SQLite competes with fopen().**

## Let's connect to our db file with SQLite

In [5]:
# Name file whatever you want. .sqlite creates file. conn creates connection
import sqlite3
sqlite_db = 'dsi-db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

## Now we'll create a table using our cursor

In [6]:
c.execute('CREATE TABLE houses \
          (field1 INTEGER PRIMARY KEY, sqft INTEGER,\
           bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes
conn.commit()

OperationalError: table houses already exists

## Now, we'll add rows to our table

In [7]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', last_sale)

# Remember to commit the changes
conn.commit()

## We can bring up DB Browser to see our results...

## Now let's bulk add mulitple rows with executemany()

In [8]:
# Execute many inserts multiple rows
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

## Now, we'll bulk add using a csv

In [9]:
pd.read_csv('https://www.dropbox.com/s/1k9cgsd7bzce0yk/housing-data.csv?dl=1').head(2)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
# check that our entries match
c.execute('PRAGMA table_info(houses)')
c.fetchall()

In [ ]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt('https://www.dropbox.com/s/1k9cgsd7bzce0yk/housing-data.csv?dl=1', dtype='i8', delimiter=',', skip_header=1)).tolist()

## Here we see what that looks like

In [ ]:
data[0:3]

## To auto-increment the PK, we need 'None', so we'll add that

In [ ]:
# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

## Here we see that result

In [ ]:
data[0:3]

## Now we'll loop through and add each

In [ ]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

## Again, we can see the results with DB Broswer

## We can also see our efforts with a query

In [ ]:
# Similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# Here results is a cursor object - use fetchall() to extract a list
results.fetchall()

## Using pandas

In [ ]:
import pandas as pd
from pandas.io import sql

In [ ]:


data = pd.read_csv('https://www.dropbox.com/s/1k9cgsd7bzce0yk/housing-data.csv?dl=1', low_memory=False)
data.head()

## Now we'll use the pandas .to_sql() method to write another table

In [ ]:
data.to_sql('houses_pandas',
            con=conn,
            if_exists='replace',
            index=False)

## Again, we can see our table using SQLite browser

## And we can query to read it

In [ ]:
sql.read_sql('select * from houses_pandas limit 5', con=conn)

## Exercise

- Create a new database file using SQLite
- Create a table in that db file called students
- Insert the names of all the people sitting at your table
- Create the table with a auto-incrementing primary key
- Add a column for their favorite color and number
- Once the table is populated use pandas to select all the data from the table

## SQL Operators

## SELECT

```SQL
SELECT
<columns>
FROM
<table>
```

## Now within pandas

In [10]:
sql.read_sql('select * from houses_pandas limit 10', con=conn)

NameError: name 'sql' is not defined

```SQL
SELECT *
```
denotes returns all of the columns.

### We can also select individual columns

```SQL
SELECT
<col1>, <col2>, <coln>
FROM
<table>
```

## And with pandas...

In [ ]:
sql.read_sql('select age, price from houses_pandas limit 10', con=conn)

## Exercise

- Write a query that returns only bedrooms, sq. footage, and price from our houses_pandas table
- Implement the query in pandas

In [ ]:

sql.read_sql('select bdrms, sqft, price from houses_pandas limit 10', con=conn)

## WHERE

### Where is used to filter the data 

```SQL
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```

### Example in SQL

```SQL
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 500000;
```

## Now, we'll execute it in pandas

In [ ]:
sql.read_sql('select sqft, bdrms, age, price from houses_pandas\
             where bdrms = 2 and price < 500000', con=conn)

## Exercise

- Write a query that returns the sqft, bdrms, age for  houses older than 60 years.
- Implement the query in pandas

In [ ]:

sql.read_sql('select sqft, bdrms, age\
             from houses_pandas where age > 60', con=conn)

## AGGREGATIONS

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

## Example SQL

```SQL
SELECT COUNT(price)
FROM houses_pandas;
```

In [ ]:
sql.read_sql('SELECT COUNT(price) FROM houses_pandas', con=conn)

## Another example

```SQL
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

In [ ]:
sql.read_sql('SELECT AVG(sqft), MIN(price), \
MAX(price) FROM houses_pandas WHERE bdrms = 2', con=conn)

## Exercise 

- Write a query to find the average price per sq ft for one bedroom houses
- Write another to find the average price per sq ft for those greater than 3 bedrooms
- Implement both in pandas sql

In [ ]:
sql.read_sql('SELECT AVG(sqft), MIN(price), \
MAX(price) FROM houses_pandas WHERE bdrms = 2', con=conn)

In [ ]:
sql.read_sql('SELECT SUM(price)/SUM(sqft) FROM houses_pandas WHERE bdrms > 3', con=conn)

In [ ]:
sql.read_sql('SELECT SUM(price)/SUM(sqft) FROM houses_pandas WHERE bdrms = 1', con=conn)

## Independent Practice

Practice querying the **PostgreSQL database** using Postico. You can find the DB at:

```
url: dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com
port: 5432
database: dsi
user: dsi_student
password: gastudents
```

Questions:

- What's the average price per room for 1 bedroom apartments?
- What's the average price per room for 2 bedrooms apartments?
- What's the most frequent apartment size (in terms of bedrooms)?
- How many are there of that apartment kind?
- What fraction of the total number are of that kind?
- How old is the oldest 3 bedrooms apartment?
- How old is the youngest apartment?
- What's the average age for the whole dataset?
- What's the average age for each bedroom size?

Try to answer all these in SQL.

If you finish, try completing the first sections of <br>[PostgreSQL Exercises](https://pgexercises.com/questions/basic/selectall.html)<br>[SQL zoo](http://www.sqlzoo.net/).

In [ ]:
sql.read_sql('SELECT AVG(price)/COUNT(bdrm) FROM houses_pandas WHERE bdrms = 1', con=conn)

## Conclusion

- We've seen how to use a SQL database and a SQLite file database to make queries
- We've seen how to create a table and populate it
- We've seen how to use pandas sql to make queries
- We've seen how to use select, where, and aggregations